<a href="https://colab.research.google.com/github/ShahrzadZolghadr/Sentiment-Analysis/blob/main/SA_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install hazm
!pip install -q clean-text[gpl]
#!pip install transformers

     |████████████████████████████████| 316 kB 4.2 MB/s 
     |████████████████████████████████| 1.4 MB 36.1 MB/s 
     |████████████████████████████████| 233 kB 44.7 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=5535b257e4b40920dabb9d2663e4124513121abdc52b6f248fb8aa7e57990ceb
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154576 sha256=8229c703cdb46547813b11a002aea89559e7374e7d901119a0f434b72cab0d9c
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
     |████████████████████████████████| 184 kB 5.7 MB/s 
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |██

In [3]:
import pandas as pd
import hazm
import re
from cleantext import clean
import os
from tqdm.notebook import tqdm
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

In [4]:
!unzip drive/MyDrive/snappfood

Archive:  drive/MyDrive/snappfood.zip
   creating: snappfood/
  inflating: __MACOSX/._snappfood    
  inflating: snappfood/test.csv      
  inflating: __MACOSX/snappfood/._test.csv  
  inflating: snappfood/dev.csv       
  inflating: __MACOSX/snappfood/._dev.csv  
  inflating: snappfood/train.csv     
  inflating: __MACOSX/snappfood/._train.csv  


In [5]:
train = pd.read_csv('snappfood/train.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
test = pd.read_csv('snappfood/test.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
dev = pd.read_csv('snappfood/dev.csv',error_bad_lines=False,sep = '\t', encoding='utf-8')
train.head()

,Unnamed: 0,comment,label,label_id
0,0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,SAD,1
1,1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,HAPPY,0
2,2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,SAD,1
3,3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,HAPPY,0
4,4,شیرینی وانیلی فقط یک مدل بود.,HAPPY,0


In [6]:
train = train[['comment', 'label_id']]
test = test[['comment', 'label_id']]
dev = dev[['comment', 'label_id']]
train.head()

,comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


## Preprocessing text column data

In [7]:
# calculate the length of comments based on their words
train['comment_len_by_words'] = train['comment'].apply(lambda t: len(hazm.word_tokenize(t)))
min_max_len = train["comment_len_by_words"].min(), train["comment_len_by_words"].max()
print(f'Min: {min_max_len[0]} \tMax: {min_max_len[1]}')

Min: 2 	Max: 378


In [8]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


def cleaning(text):
    text = text.strip()
    
    # regular cleaning
    text = clean(text,
        fix_unicode=True,
        to_ascii=False,
        lower=True,
        no_line_breaks=True,
        no_urls=True,
        no_emails=True,
        no_phone_numbers=True,
        no_numbers=False,
        no_digits=False,
        no_currency_symbols=True,
        no_punct=False,
        replace_with_url="",
        replace_with_email="",
        replace_with_phone_number="",
        replace_with_number="",
        replace_with_digit="0",
        replace_with_currency_symbol="",
    )

    # cleaning htmls
    text = cleanhtml(text)
    
    # normalizing
    normalizer = hazm.Normalizer()
    text = normalizer.normalize(text)
    
    # removing wierd patterns
    wierd_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
        u"\u2069"
        u"\u2066"
        # u"\u200c"
        u"\u2068"
        u"\u2067"
        "]+", flags=re.UNICODE)
    
    text = wierd_pattern.sub(r'', text)
    
    # removing extra spaces, hashtags
    text = re.sub("#", "", text)
    text = re.sub("\s+", " ", text)
    
    return text

In [9]:
# cleaning comments
train['cleaned_comment'] = train['comment'].apply(cleaning)
test['cleaned_comment'] = test['comment'].apply(cleaning)
dev['cleaned_comment'] = dev['comment'].apply(cleaning)

train.head()

,comment,label_id,comment_len_by_words,cleaned_comment
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1,9,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0,28,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1,19,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0,21,عالللی بود همه چه درست و به اندازه و کیفیت خوب...
4,شیرینی وانیلی فقط یک مدل بود.,0,7,شیرینی وانیلی فقط یک مدل بود.


In [10]:
train = train[['cleaned_comment', 'label_id']]
test = test[['cleaned_comment', 'label_id']]
dev = dev[['cleaned_comment', 'label_id']]
train.head()

,cleaned_comment,label_id
0,واقعا حیف وقت که بنویسم سرویس دهیتون شده افتضاح,1
1,قرار بود ۱ ساعته برسه ولی نیم ساعت زودتر از مو...,0
2,قیمت این مدل اصلا با کیفیتش سازگاری نداره، فقط...,1
3,عالللی بود همه چه درست و به اندازه و کیفیت خوب...,0
4,شیرینی وانیلی فقط یک مدل بود.,0


In [11]:
x_train, y_train = train['cleaned_comment'].values.tolist(), train['label_id'].values.tolist()
x_test, y_test = test['cleaned_comment'].values.tolist(), test['label_id'].values.tolist()
x_dev, y_dev = dev['cleaned_comment'].values.tolist(), dev['label_id'].values.tolist()
print(train.shape)

(56700, 2)


# Keras

In [12]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

max_features = 15000  # Only consider the top 15k words
maxlen = 200  # Only consider the first 200 words of each movie review

In [13]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)

X_train = tokenizer.texts_to_sequences(x_train)
X_test = tokenizer.texts_to_sequences(x_test)
X_dev = tokenizer.texts_to_sequences(x_dev)

X_train = pad_sequences(X_train, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)
X_dev = pad_sequences(X_dev, maxlen=maxlen)

In [14]:
type(y_dev[0])

int

# Build the model

In [15]:
# Input for variable-length sequences of integers
inputs = keras.Input(shape=(None,), dtype="int32")
# Embed each integer in a 128-dimensional vector
x = layers.Embedding(max_features, 128)(inputs)
# Add 2 bidirectional LSTMs
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
# Add a classifier
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 128)         1920000   
_________________________________________________________________
bidirectional (Bidirectional (None, None, 128)         98816     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 128)               98816     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,117,761
Trainable params: 2,117,761
Non-trainable params: 0
_________________________________________________________________


In [16]:
model.compile("adam", "binary_crossentropy", metrics=["accuracy"])
model.fit(X_train, np.array(y_train), batch_size=32, epochs=2, validation_data=(X_dev, np.array(y_dev)))

Epoch 1/2
1772/1772 [==============================] - 351s 192ms/step - loss: 0.3691 - accuracy: 0.8391 - val_loss: 0.3434 - val_accuracy: 0.8541
Epoch 2/2
1772/1772 [==============================] - 338s 191ms/step - loss: 0.2891 - accuracy: 0.8795 - val_loss: 0.3451 - val_accuracy: 0.8560


In [17]:
# Evaluate the model on the test data using `evaluate`
print("Evaluate on test data")
results = model.evaluate(X_test, np.array(y_test), batch_size=32)
print("test loss, test acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
print("Generate predictions for 3 samples")
predictions = model.predict(X_test[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
219/219 [==============================] - 17s 76ms/step - loss: 0.3345 - accuracy: 0.8594
test loss, test acc: [0.33447641134262085, 0.8594285845756531]
Generate predictions for 3 samples
predictions shape: (3, 1)


In [24]:
model.predict(pad_sequences(tokenizer.texts_to_sequences([['چقددددر آخه بیمزه باشه یه غذا']]),maxlen))

array([[0.36567947]], dtype=float32)

In [25]:
predictions

array([[0.98221475],
       [0.04858767],
       [0.7933992 ]], dtype=float32)